In [ ]:
!pip install datasets transformers evaluate optuna
!apt-get install git-lfs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 r

In [ ]:
from datasets import load_dataset
imdb = load_dataset("imdb")
print(imdb)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
train_dataset = imdb['train'].shuffle(seed=42)
test_dataset = imdb['test'].shuffle(seed=42)

# For hyperparameter tuning
val_split_ratio = 0.3
val_size = int(len(imdb["train"]) * val_split_ratio)

val_dataset_ht = train_dataset.select([i for i in list(range(val_size))])
train_dataset_ht = train_dataset.select([i for i in list(range(val_size, len(imdb["train"])))])

print(f"Full train dataset size: {len(train_dataset)}")
print(f"Validation dataset size for hyperparameter tuning: {len(val_dataset_ht)}")
print(f"Train dataset size for hyperparameter tuning: {len(train_dataset_ht)}")

Full train dataset size: 25000
Validation dataset size for hyperparameter tuning: 7500
Train dataset size for hyperparameter tuning: 17500


In [ ]:
import torch
import optuna
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import evaluate

torch.manual_seed(42)

# Load BERT tokenizer - using base uncased model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    # BERT has a max sequence length of 512
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

train_dataset_ht = train_dataset_ht.map(tokenize_function, batched=True)
val_dataset_ht = val_dataset_ht.map(tokenize_function, batched=True)

# Format datasets for the model
train_dataset_ht.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset_ht.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/17500 [00:00<?, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [ ]:
# Hyperparameter tuning with Optuna
def objective(trial: optuna.Trial):
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    # Define the hyperparameters to tune
    batch_size = trial.suggest_categorical('batch_size', [16, 32])
    learning_rate = trial.suggest_categorical('learning_rate', [5e-5, 3e-5, 2e-5])
    num_epochs = trial.suggest_categorical('num_epochs', [2, 3, 4])
    weight_decay = trial.suggest_categorical('weight_decay', [0.01, 0.1])

    training_args = TrainingArguments(
        output_dir=f"./results/trial_{trial.number}",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=weight_decay,
        logging_dir=f"./logs/trial_{trial.number}",
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_ht,
        eval_dataset=val_dataset_ht
    )

    trainer.train()

    # Evaluate the model on the validation dataset
    eval_result = trainer.evaluate()

    # Return validation loss
    return eval_result["eval_loss"]

# Run Optuna study
print("Starting hyperparameter tuning...")
study = optuna.create_study(study_name='hp-search-bert', direction='minimize')
study.optimize(func=objective, n_trials=5)

best_lr = float(study.best_params['learning_rate'])
best_batch_size = study.best_params['batch_size']
best_epoch = int(study.best_params['num_epochs'])
best_weight_decay = float(study.best_params['weight_decay'])

print(f"Best Learning Rate: {best_lr}")
print(f"Best Batch Size: {best_batch_size}")
print(f"Best Epochs: {best_epoch}")
print(f"Best Weight Decay: {best_weight_decay}")

[I 2025-04-08 03:30:02,716] A new study created in memory with name: hp-search-bert


Starting hyperparameter tuning...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

Epoch,Training Loss,Validation Loss
1,0.222700,0.228407
2,0.136400,0.291581
3,0.035300,0.340204


[I 2025-04-08 04:52:08,297] Trial 3 finished with value: 0.22840693593025208 and parameters: {'batch_size': 16, 'learning_rate': 5e-05, 'num_epochs': 3, 'weight_decay': 0.01}. Best is trial 1 with value: 0.18661415576934814.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.203100,0.209678
2,0.128500,0.242661
3,0.069100,0.280605
4,0.029200,0.322975


[I 2025-04-08 05:17:04,172] Trial 4 finished with value: 0.20967787504196167 and parameters: {'batch_size': 32, 'learning_rate': 3e-05, 'num_epochs': 4, 'weight_decay': 0.1}. Best is trial 1 with value: 0.18661415576934814.


Best Learning Rate: 3e-05
Best Batch Size: 16
Best Epochs: 3
Best Weight Decay: 0.1


In [ ]:
# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='binary')["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average='binary')["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average='binary')["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}

# Initialize the BERT model for the final training
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Process the full datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Format datasets for the model
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Set up training arguments with best hyperparameters
training_args = TrainingArguments(
    output_dir="./results/bert_final",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch,
    weight_decay=best_weight_decay,
    logging_dir="./logs/bert_final",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="tensorboard"
)

# Initialize Trainer for final training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
print("Starting final training...")
train_results = trainer.train()
print(train_results)

# Evaluate the model on test set
print("Evaluating on test set...")
eval_results = trainer.evaluate()
print(eval_results)

# Save the final model
model_path = "./bert_imdb_sentiment"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model saved to {model_path}")

Starting final training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,0.224800,0.227690,0.924760,0.927740,0.966000,0.892395
2,0.126800,0.225532,0.939120,0.939866,0.951520,0.928493
3,0.054000,0.274605,0.939640,0.939773,0.941840,0.937714


TrainOutput(global_step=4689, training_loss=0.14637515213881558, metrics={'train_runtime': 1906.6215, 'train_samples_per_second': 39.337, 'train_steps_per_second': 2.459, 'total_flos': 1.9733329152e+16, 'train_loss': 0.14637515213881558, 'epoch': 3.0})
Evaluating on test set...


{'eval_loss': 0.2746053636074066, 'eval_accuracy': 0.93964, 'eval_f1': 0.9397725004989024, 'eval_recall': 0.94184, 'eval_precision': 0.9377140581441656, 'eval_runtime': 152.1002, 'eval_samples_per_second': 164.365, 'eval_steps_per_second': 10.276, 'epoch': 3.0}
Model saved to ./bert_imdb_sentiment
